In [2]:
import torch
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

In [8]:
trainset = pd.read_csv("processed_taxi/train.csv")
testset = pd.read_csv("processed_taxi/test.csv")
testset.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
split_col = "TAXI_ID"
split_col_len = 981

In [5]:
import pickle
models = [pickle.load(open("models/"+str(i), 'rb')) for i in range(split_col_len)]

In [9]:
x_test = testset.drop(columns=["TRIP_ID", "TIMESTAMP"])
outputs = [max(0, models[x_test[split_col][i]].predict(torch.tensor(row).reshape(1, -1))[0]) for i, row in x_test.iterrows()]
t = np.array([])
final_predictions = pd.DataFrame().assign(
    TRIP_ID= testset["TRIP_ID"],
    TRAVEL_TIME = outputs
    )

out_mean = final_predictions["TRAVEL_TIME"].mean()
act_mean = trainset["LEN"].mean()

final_predictions["TRAVEL_TIME"] = final_predictions["TRAVEL_TIME"].apply(lambda x : x - out_mean + act_mean)

final_predictions

,TRIP_ID,TRAVEL_TIME
0,T1,880.104702
1,T2,723.233811
2,T3,684.931814
3,T4,472.664370
4,T5,661.324974
...,...,...
315,T323,603.669911
316,T324,551.229078
317,T325,893.153215
318,T326,637.141469


In [10]:
final_predictions.to_csv("final_predictions.csv", index=False)